In [ ]:
#mainファイル

In [ ]:
import glob
import tkinter
from tkinter import filedialog
from PIL import Image, ImageFont, ImageDraw, ImageOps

In [ ]:
import time

In [ ]:
import shutil
import os
#if os.path.isdir("./out_image"):
#    shutil.rmtree("./out_image")
#if os.path.isdir("./out_debug"):
#    shutil.rmtree("./out_debug")

In [ ]:
#座標
grid={
    #left,top,right,bottom
    "all":[1515,136,1882,925],
    "change":[1515,296,1882,549],
    "skill_1":[1546,397,1850,431],
    "skill_2":[1546,471,1850,505],
    "skill_1_lv":[1857,439,1877,465],
    "skill_2_lv":[1857,515,1877,541],
    "slot_1":[1748,299,1791,339],
    "slot_2":[1791,299,1834,339],
    "slot_3":[1834,299,1877,339]
}

In [ ]:
root=tkinter.Tk()
root.attributes('-topmost', True)
root.withdraw()
#fTyp = [("",".png .jpg .mp4")]
fTyp = [("",".jpg .mp4")]
iDir = os.path.abspath(os.path.dirname("__file__"))
files = list(filedialog.askopenfilenames(filetypes = fTyp,initialdir = iDir))
for i in files:
    print("load:"+i)

In [ ]:
#出力フォルダ作成
#ディレクトリ確認と作成
if not os.path.isdir("./out_image"):
    os.mkdir("./out_image")
for g in grid:
    if not os.path.isdir("./out_image"+"/"+g):
        os.mkdir("./out_image"+"/"+g)

In [ ]:
def image_crop(image,grid_name):
    """Imageオブジェクトの画像をgrid_nameの座標で切り取って保存するコード"""
    
    image_crop = image.crop(grid[grid_name])  
    width, height = image_crop.size
    image_crop = image_crop.resize((width*2, height*2))
    image_crop_gray = image_crop.convert("L")  
    #dirnum=len(os.listdir("./out_image/"+grid_name))+1
    #image_crop_gray.save("./out_image/"+grid_name+"/"+str(dirnum)+".jpg",quality=95)
    return image_crop_gray

In [ ]:
#ここから本格的に作る

In [ ]:
#最終出力
def first_frame():
    out_param={
        "skill_1":"",
        "skill_1_lv":"",
        "skill_2":"",
        "skill_2_lv":"0",
        "slot_1":"0",
        "slot_2":"0",
        "slot_3":"0",
    }
    return out_param

In [ ]:
#エラー出力用
def error_dic_set():
    error_dic={
        "skill_1":"",
        "skill_1_lv":"",
        "skill_2":"",
        "skill_2_lv":"",
        "slot_1":"0",
        "slot_2":"0",
        "slot_3":"0",
        "path":"",
        "path2":"",
        "full_path":"",
    }
    return error_dic

In [ ]:
import sys
from PIL import Image
import pyocr
import pyocr.builders
import tkinter
from tkinter import filedialog

In [ ]:
#OCRの準備
pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    #sys.exit(1)

tool = tools[0]

print("Will use tool '%s'" % (tool.get_name()))

langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))

In [ ]:
def ocr_skill(image_ocr):
    """スキル1とスキル2"""
    pred = tool.image_to_string(
        image_ocr,
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=7)#7は画像を1行のテキストとみなす
    )
    return pred

In [ ]:
def ocr_skill_lv(image_ocr):
    """スキル1とスキル2のレベルocr版"""
    pred = tool.image_to_string(
        image_ocr,
        lang="eng",
        builder=pyocr.builders.TextBuilder(tesseract_layout=10)#10は画像を1文字とみなす
    )
    return pred

In [ ]:
#スロット数判定の準備
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow as tf
from tensorflow.keras.preprocessing import image as tf_image
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
slot_model = tf.keras.models.load_model('./data/model/slot_model.h5')
slot_model.summary()

In [ ]:
def classifier_slot(img):
    """スロットのレベル"""
    x = np.array(img.resize((80,86)).convert("RGB"))
    x = np.expand_dims(x, axis=0)
    pred = slot_model.predict(x)[0]
    pred_list=pred.tolist()
    return pred_list.index(max(pred))

In [ ]:
#ocrの誤字訂正
ocr_skill_words={
    "プレ抑制":"ブレ抑制",
    "力の解放":"力の開放",
    "ポマー":"ボマー",
    "ポボマー":"ボマー",
    "フルチヤージ":"フルチャージ",
    "フルチャヤージ":"フルチャージ",
    "たる":"フルチャージ",
    "火事場カ":"火事場力",
    "炊事場カ":"火事場力",
    "炎事場カ":"火事場力",
    "泡涼の舞":"泡沫の舞",
    "泡洲の舞":"泡沫の舞",
    "ひるの軽減":"ひるみ軽減",
    "ひるのみの軽減":"ひるみ軽減",
    "逆肌":"逆襲",
    "逆要":"逆襲",
    "逆柚":"逆襲",
    "澤身":"渾身",
    "源身":"渾身",
    "スタミナ怠速回復":"スタミナ急速回復",
    "スタダミナ怠速回復":"スタミナ急速回復",
    "スタダミナ急速回復":"スタミナ急速回復",
    "スタミナ礁取":"スタミナ奪取",
    "スタミナ容取":"スタミナ奪取",
    "スタミナ秦取":"スタミナ奪取",
    "スタミナ内取":"スタミナ奪取",
    "スタミナ審取":"スタミナ奪取",
    "耐居":"耐震",
    "耐岩":"耐震",
    "耐展":"耐震",
    "耳析":"耳栓",
    "耳柏":"耳栓",
    "耳要":"耳栓",
    "耳梓":"耳栓",
    "耳欄":"耳栓",
    "たコ":"破壊王",
    "永耐性":"氷耐性",
    "鳳圧耐性":"風圧耐性",
    "レルアルエニ":"ランナー",
    "レアルエニ":"ランナー",
    "レルアルエ":"ランナー",
    "レルム":"ランナー",
    "レアルム":"ランナー",
    "レルル":"ランナー",
    "翔政使い":"翔蟲使い",
    "翔蝶使い":"翔蟲使い",
    "翔副使い":"翔蟲使い",
    "翔悪使い":"翔蟲使い",
    "翔了使い":"翔蟲使い",
    "翔政使い":"翔蟲使い",
    "翔軸使い":"翔蟲使い",
    "翔工使い":"翔蟲使い",
    "翔識使い":"翔蟲使い",
    "翔職使い":"翔蟲使い",
    "翔職人使い":"翔蟲使い",
    "にさり川目":"精霊の加護",
    "にさり川上":"精霊の加護",
    "にの川目":"精霊の加護",
    "ドコの凡中":"精霊の加護",
    "にの川上":"精霊の加護",
    "精寺の加護":"精霊の加護",
    "精土の加護":"精霊の加護",
    "精填の加護":"精霊の加護",
    "精露の加護":"精霊の加護",
    "ビュ":"地質学",
    "ヒキュ":"植生学",
    "にキュ":"植生学",
    "査属性攻撃強化":"龍属性攻撃強化",
    "次属性攻撃強化":"火属性攻撃強化",
    "上エア":"乗り名人",
    "上アミ":"乗り名人",
    "エや":"乗り名人",
    "上や":"乗り名人",
    "エヤミ":"乗り名人",
    "上ヤミ":"乗り名人",
    "EDMEH":"毒耐性",
    "EDMEd":"毒耐性",
    "キッ":"業物",
    "キル":"業物",
    "証キッ":"業物",
    "証キリ":"業物",
    "ムウルアルンタ":"ジャンプ鉄人",
    "にすシダミ":"剥ぎ取り鉄人",
    "にすりシタ":"剥ぎ取り鉄人",
    "泥寺耐性":"泥雪耐性",
    "氏吹き名人":"笛吹き名人",
    "仙吹き名人":"笛吹き名人",
    "穫吹き名人":"笛吹き名人",
    "守心撃【属性】":"会心撃【属性】",
    "貫通弾・賃通矢強化":"貫通弾・貫通矢強化",
    "貫通弾・貨通矢強化":"貫通弾・貫通矢強化",
    "ガード綴化":"ガード強化",
    "晶食い":"早食い",
    "区":"匠",
    "体力回復時UP":"体力回復量UP",
    "体力回復旦UP":"体力回復量UP",
    "":"",
    "":"",

}

#ocrのstr内の文字置換(例:半角スペース→削除)
ocr_skill_editwords={#KOUP以外はでない
    " ":"",
    "|":"",
    ":":"",
    "0":"O",
    "1":"",
    "2":"",
    "3":"",
    "4":"",
    "5":"",
    "6":"",
    "7":"",
    "8":"",
    "9":"",
    "A":"",
    "B":"",
    "C":"",
    "D":"",
    "E":"",
    "F":"",
    "G":"",
    "H":"",
    "I":"",
    "J":"",
    #"K":"",
    "L":"",
    "M":"",
    "N":"",
    #"O":"",
    #"P":"",
    "Q":"",
    "R":"",
    "S":"",
    "T":"",
    #"U":"",
    "V":"",
    "W":"",
    "X":"",
    "Y":"",
    "Z":"",
    "o":"O",
    "k":"K",
    "(":"",
    ")":"",
    "{":"",
    "}":"",
    "[":"",
    "]":"",
    "":"",
    "":"",

}

ocr_skill_lv_words={
    "l":1,
    "i":1,
    "I":1,
    "|":1,
    "]":1,
    "[":1,
    "S":2,
    "7":2,
    "a":2,
}

ocr_skill_lv_editwords={
    " ":"",
}

skill_lv_class=[1,2,3,4,5]




In [ ]:
#全スキル読み込み
import pandas as pd
all_skill_path="./all_skill.xlsx"
df = pd.read_excel(all_skill_path, sheet_name=0, header=0, index_col=None)

In [ ]:
def skill_tf(name):#スキル名がxlsxファイルに入っているかの確認0or1
    return df[df["skill_name"]==name]["skill_name"].count()

In [ ]:
def image_niti(image_fanc_niti,sikii):#sikii未満は0、以上はそのまま
    return ImageOps.invert(image_fanc_niti.point(lambda x: 0 if x < sikii else x))

In [ ]:
#if not os.path.isdir("./out_debug"):
#    os.mkdir("./out_debug")
#if not os.path.isdir("./out_debug/skill"):
#    os.mkdir("./out_debug/skill")
#if not os.path.isdir("./out_debug/skill_lv"):
#    os.mkdir("./out_debug/skill_lv")
#for skill in list(df["skill_name"]):
#    if not os.path.isdir("./out_debug/skill/"+skill):
#        os.mkdir("./out_debug/skill/"+skill)
#if not os.path.isdir("./out_debug/skill/スキル無し"):
#    os.mkdir("./out_debug/skill/スキル無し")
#for lv in [0,1,2,3,4]:
#    if not os.path.isdir("./out_debug/skill_lv/"+str(lv)):
#        os.mkdir("./out_debug/skill_lv/"+str(lv))

In [ ]:
#main
#各defの下必須
#debugどのくらいスキルとスキルlvが合っているかの確認
def main(image):#Imageオブジェクト入れたらparamの辞書型が帰ってくる    
    out_param=first_frame()#param初期値
    error_dic=error_dic_set()#error初期値
    error_flug=False
    
    #skill_1
    skill_1=ocr_skill(image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 100 else x)),"skill_1"))
    for skill_edit in ocr_skill_editwords:
        skill_1=skill_1.replace(skill_edit,ocr_skill_editwords[skill_edit])
    if skill_1 in ocr_skill_words:
        skill_1=ocr_skill_words[skill_1]
    if skill_tf(skill_1)==0:#スキル一覧にスキルがなかった場合
        print("Not found skill_1:"+skill_1)
        error_dic["skill_1"]=skill_1
        
        dirnum=len(os.listdir("./out_image/"+"skill_1"))+1
        image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_1").save("./out_image/"+"skill_1"+"/"+str(dirnum)+".jpg",quality=95)
        crop_path="./out_image/"+"skill_1"+"/"+str(dirnum)+".jpg"
        error_dic["path"]="=HYPERLINK(\""+crop_path+"\")"

        dirnum_c=len(os.listdir("./out_image/"+"change"))+1
        image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
        crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
        error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"

        error_flug=True
        #return 0,error_dic
    else:#スキル一覧にスキルがあった場合
        out_param["skill_1"]=skill_1
        #dirnum_skill=len(os.listdir("./out_debug/skill/"+skill_1))+1
        #shutil.copy2(crop_path, "./out_debug/skill/"+skill_1+"/"+str(dirnum_skill)+".jpg")
        

    
    #skill_1_lv
    #crop_path=image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_1_lv")
    skill_1_lv=ocr_skill_lv(image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_1_lv"))
    for skill_lv_edit in ocr_skill_lv_editwords:
        skill_1_lv=skill_1_lv.replace(skill_lv_edit,ocr_skill_lv_editwords[skill_lv_edit])
    if skill_1_lv in ocr_skill_lv_words:
        skill_1_lv=ocr_skill_lv_words[skill_1_lv]
    try:skill_1_lv=int(skill_1_lv)#intに変換できるか検証
    except:
        print("Not found skill_1_lv:"+skill_1_lv)
        error_dic["skill_1"]=skill_1
        error_dic["skill_1_lv"]=skill_1_lv

        dirnum_=len(os.listdir("./out_image/"+"skill_1_lv"))+1
        image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_1_lv").save("./out_image/"+"skill_1_lv"+"/"+str(dirnum_)+".jpg",quality=95)
        crop_path_="./out_image/"+"skill_1_lv"+"/"+str(dirnum_)+".jpg"
        error_dic["path"]="=HYPERLINK(\""+crop_path_+"\")"

        dirnum_c=len(os.listdir("./out_image/"+"change"))+1
        image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
        crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
        error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"

        error_flug=True
        #return 0,error_dic

    if skill_1_lv in skill_lv_class:#スキルレベルが1~5の場合
        out_param["skill_1_lv"]=skill_1_lv
        #dirnum_skill_lv=len(os.listdir("./out_debug/skill_lv/"+str(skill_1_lv)))+1
        #shutil.copy2(crop_path_, "./out_debug/skill_lv/"+str(skill_1_lv)+"/"+str(dirnum_skill_lv)+".jpg")
    else:
        print("Over skill_1_lv:"+str(skill_1_lv))
        error_dic["skill_1"]=skill_1
        error_dic["skill_1_lv"]=skill_1_lv
        error_dic["path"]="=HYPERLINK(\""+crop_path_+"\")"

        dirnum_c=len(os.listdir("./out_image/"+"change"))+1
        image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
        crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
        error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"

        error_flug=True
        #return 0,error_dic


    
    #skill_2
    skill_2_nothing_flug=False
    skill_2_no_flug=False

    #crop_path=image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 100 else x)),"skill_2")
    crop_image=image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 100 else x)),"skill_2")
    skill_2=ocr_skill(crop_image)
    for skill_edit in ocr_skill_editwords:
        skill_2=skill_2.replace(skill_edit,ocr_skill_editwords[skill_edit])
    if skill_2 in ocr_skill_words:
        skill_2=ocr_skill_words[skill_2]
    if np.sum(255-np.array(crop_image, dtype=np.uint8))<=5000:#ほぼ真っ白の場合
        pass
    else:
        if skill_tf(skill_2)==0:#スキル一覧にスキルがなかった場合
            skill_2_nothing_flug=True#スキルなしかもしれないからフラグ立てとく
            #print("Not found skill_2:"+skill_2)
            #return 0
        else:#スキル一覧にスキルがあった場合
            out_param["skill_2"]=skill_2
            #dirnum_skill=len(os.listdir("./out_debug/skill/"+skill_2))+1
            #shutil.copy2(crop_path, "./out_debug/skill/"+skill_2+"/"+str(dirnum_skill)+".jpg")


        #skill_2_lv
        #crop_path_=image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2_lv")
        skill_2_lv=ocr_skill_lv(image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2_lv"))
        for skill_lv_edit in ocr_skill_lv_editwords:
            skill_2_lv=skill_2_lv.replace(skill_lv_edit,ocr_skill_lv_editwords[skill_lv_edit])
        if skill_2_lv in ocr_skill_lv_words:
            skill_2_lv=ocr_skill_lv_words[skill_2_lv]
        try:skill_2_lv=int(skill_2_lv)#intに変換できるか検証
        except:
            if skill_2_nothing_flug:#名前も検出できないしレベルも検出できなかった場合
                #スキル2なし
                pass
                skill_2_no_flug=True
                #print("Not found skill_2:"+skill_2)
            else:
                print("Not found skill_2_lv:"+skill_2_lv)
                error_dic["skill_1"]=skill_1
                error_dic["skill_1_lv"]=skill_1_lv
                error_dic["skill_2"]=skill_2
                error_dic["skill_2_lv"]=skill_2_lv

                dirnum_=len(os.listdir("./out_image/"+"skill_2_lv"))+1
                image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2_lv").save("./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg",quality=95)
                crop_path_="./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg"
                error_dic["path"]="=HYPERLINK(\""+crop_path_+"\")"

                dirnum_c=len(os.listdir("./out_image/"+"change"))+1
                image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
                crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
                error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"

                error_flug=True
                #return 0,error_dic
        
        if not skill_2_no_flug:#スキル2なし確定以外の場合
            if skill_2_lv in skill_lv_class:#スキルレベルが1~5の場合
                if skill_2_nothing_flug:#名前が検出できなかった場合
                    print("Not found skill_2:"+skill_2)
                    error_dic["skill_1"]=skill_1
                    error_dic["skill_1_lv"]=skill_1_lv
                    error_dic["skill_2"]=skill_2
                    error_dic["skill_2_lv"]=skill_2_lv

                    dirnum=len(os.listdir("./out_image/"+"skill_2"))+1
                    image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2").save("./out_image/"+"skill_2"+"/"+str(dirnum)+".jpg",quality=95)
                    crop_path="./out_image/"+"skill_2"+"/"+str(dirnum)+".jpg"

                    dirnum_=len(os.listdir("./out_image/"+"skill_2_lv"))+1
                    image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2_lv").save("./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg",quality=95)
                    crop_path_="./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg"

                    error_dic["path"]="=HYPERLINK(\""+crop_path+"\")"
                    error_dic["path2"]="=HYPERLINK(\""+crop_path_+"\")"

                    dirnum_c=len(os.listdir("./out_image/"+"change"))+1
                    image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
                    crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
                    error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"

                    error_flug=True
                    #return 0,error_dic

                out_param["skill_2_lv"]=skill_2_lv
                #dirnum_skill_lv=len(os.listdir("./out_debug/skill_lv/"+str(skill_2_lv)))+1
                #shutil.copy2(crop_path_, "./out_debug/skill_lv/"+str(skill_2_lv)+"/"+str(dirnum_skill_lv)+".jpg") 
            else:
                dirnum=len(os.listdir("./out_image/"+"skill_2"))+1
                image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2").save("./out_image/"+"skill_2"+"/"+str(dirnum)+".jpg",quality=95)
                crop_path="./out_image/"+"skill_2"+"/"+str(dirnum)+".jpg"

                dirnum_=len(os.listdir("./out_image/"+"skill_2_lv"))+1
                image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"skill_2_lv").save("./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg",quality=95)
                crop_path_="./out_image/"+"skill_2_lv"+"/"+str(dirnum_)+".jpg"

                dirnum_c=len(os.listdir("./out_image/"+"change"))+1
                image_crop(ImageOps.invert(image.point(lambda x: 0 if x < 60 else x)),"change").save("./out_image/"+"change"+"/"+str(dirnum_c)+".jpg",quality=95)
                crop_path_c="./out_image/"+"change"+"/"+str(dirnum_c)+".jpg"
                error_dic["full_path"]="=HYPERLINK(\""+crop_path_c+"\")"
                if skill_2_nothing_flug:#名前も検出できないしレベルも検出できたけどoverした場合
                    print("Not found skill_2:"+skill_2)
                    error_dic["skill_1"]=skill_1
                    error_dic["skill_1_lv"]=skill_1_lv
                    error_dic["skill_2"]=skill_2
                    error_dic["skill_2_lv"]=str(skill_2_lv)
                    error_dic["path"]="=HYPERLINK(\""+crop_path+"\")"
                    error_dic["path2"]="=HYPERLINK(\""+crop_path_+"\")"
                    error_flug=True
                    #return 0,error_dic
                else:
                    print("Over skill_2_lv:"+str(skill_2_lv))
                    error_dic["skill_1"]=skill_1
                    error_dic["skill_1_lv"]=skill_1_lv
                    error_dic["skill_2"]=skill_2
                    error_dic["skill_2_lv"]=skill_2_lv
                    error_dic["path"]="=HYPERLINK(\""+crop_path_+"\")"
                    error_flug=True
                    #return 0,error_dic

    
    #slot_1
    slot_1=classifier_slot(image_crop(image,"slot_1"))
    if int(slot_1)==0:#スロットレベル0
        pass#終了してparamをリターン
    else:
        if error_flug:error_dic["slot_1"]=slot_1
        else:out_param["slot_1"]=slot_1


        #slot_2
        slot_2=classifier_slot(image_crop(image,"slot_2"))
        if int(slot_2) in [0,3]:#スロットレベル0or3
            pass#終了してparamをリターン
        else:
            if error_flug:error_dic["slot_2"]=slot_2
            else:out_param["slot_2"]=slot_2


            #slot_3
            slot_3=classifier_slot(image_crop(image,"slot_3"))
            if int(slot_3) in [0,2,3]:#スロットレベル0or2or3
                pass#終了してparamをリターン
            else:
                if error_flug:error_dic["slot_3"]=slot_3
                else:out_param["slot_3"]=slot_3
    

    #dirnum_all=len(os.listdir("./out_image/all"))+1
    #image.save("./out_image/all/"+str(dirnum_all)+".jpg",quality=95)
    
    if error_flug:
        for i in out_param:      
            if out_param[i]=="" or out_param[i]=="0":
                pass
            else:error_dic[i]=out_param[i] 
        return 0,error_dic
    else:return 1,out_param

In [ ]:
#出力用csv作っておく
import csv
import cv2
import copy
import numpy as np

In [ ]:
if not os.path.exists('./result.csv'):#出力用csvの作成
    with open('./result.csv','w',newline="") as f:
        writer = csv.writer(f)
        writer.writerow(list(first_frame().keys()))

if not os.path.exists('./error.csv'):#エラー出力用csvの作成
    with open('./error.csv','w',newline="") as f:
        writer = csv.writer(f)
        writer.writerow(list(error_dic_set().keys()))

In [ ]:
start = time.time()

for path_count,path in enumerate(files,1):
    print(str(path_count)+"/"+str(len(files))+"\t"+path)
    filename,ext=os.path.splitext(path)


    if ext==".jpg":#画像の時
        image=Image.open(path)
        image = image.resize((1920, 1080))
        result_tf,result=main(image)
        if result_tf:#成功した時
            with open('./result.csv','a',newline="") as f:
                writer = csv.DictWriter(f, list(result))
                writer.writerow(result)
        else:#途中でエラーが起きた場合
            with open('./error.csv','a',newline="") as f:
                writer = csv.DictWriter(f, list(result))
                writer.writerow(result)


    if ext==".mp4":#動画の時
        mov_count=0
        vid = cv2.VideoCapture(path)
        if not vid.isOpened():
            raise IOError("Couldn't open video")
        video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
        video_fps       = vid.get(cv2.CAP_PROP_FPS)
        video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                            int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        str_frame_num = str(int(vid.get(cv2.CAP_PROP_FRAME_COUNT)))

        image_compare_origin=Image.open("./data/compare/786.jpg")#比較用の初期画像
        image_compare_origin = image_compare_origin.convert("L")
        image_compare=copy.deepcopy(image_compare_origin)
        #if not os.path.isdir("./out_image/mov"):
        #    os.mkdir("./out_image/mov")
        frame_count=0
        while True:
            return_value, frame = vid.read()
            if frame is None:
                break
            frame_count+=1
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            image = Image.fromarray(frame)
            image = image.resize((1920, 1080))
            image_crop_gray_ = image.convert("L")#gray
            image_crop_ = image_crop_gray_.crop(grid["change"])
            image_niti_=image_niti(image_crop_,80)
            #装備なし画面を除く
            result_1=np.sum(np.power(np.asarray(image_compare_origin)-np.asarray(image_niti_),2))
            if 1000000<=result_1<=1500000:#しきい値
                result_2=np.sum(np.power(np.asarray(image_compare)-np.asarray(image_niti_),2))
                image_compare=copy.deepcopy(image_niti_)
                if 0<=result_2<=50000:#しきい値
                    pass
                else:
                    #フレーム切り替え検出したときのコード
                    mov_count+=1
                    print("movcount:"+str(mov_count)+"\t"+"frame:"+str(frame_count)+"/"+str_frame_num)
                    #if not os.path.isdir("./out_image/mov"):
                    #    os.mkdir("./out_image/mov")
                    #dirnum_mov=len(os.listdir("./out_image/mov/"))+1
                    #image.save("./out_image/mov/"+str(dirnum_mov)+".jpg",quality=95)
                    result_tf,result=main(image)

                    if result_tf:#成功した時
                        with open('./result.csv','a',newline="") as f:
                            writer = csv.DictWriter(f, list(result))
                            writer.writerow(result)
                    else:#途中でエラーが起きた場合
                        with open('./error.csv','a',newline="") as f:
                            writer = csv.DictWriter(f, list(result))
                            writer.writerow(result)
                    
            else:#ぜんぜん違う画像の場合
                pass

print("completed!!!")

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
#メモリ解放
tf.keras.backend.clear_session()

In [ ]:
#被り除去
with open('./result.csv') as f:
    reader = csv.reader(f)
    l = [row for row in reader]
with open('./result.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    writer.writerow(l[0])
    writer.writerows(list(map(list, set(map(tuple, l[1:])))))